In [1492]:
import warnings
warnings.filterwarnings('ignore')

In [1493]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import re

from simpledbf import Dbf5

from tqdm import tqdm

In [1494]:
res_22 = pd.read_csv('data/state_g22_sov_data_by_g22_srprec.csv')
res_22.head()

,COUNTY,FIPS,SRPREC,ELECTION,SRPREC_KEY,GEO_TYPE,ADDIST,CDDIST,SDDIST,BEDIST,...,SOSDEM01,SOSREP01,SPINOP01,SPINOP02,TRSDEM01,TRSREP01,USPDEM01,USPREP01,USSDEM01,USSREP01
0,1,6001,200100,g22,06001200100,srprec,14,12,7,2,...,2419,141,212,2035,2360,179,2415,132,2421,134
1,1,6001,200200,g22,06001200200,srprec,14,12,7,2,...,1516,100,113,1235,1470,115,1506,108,1520,104
2,1,6001,201400,g22,06001201400,srprec,14,12,7,2,...,2348,138,168,1965,2276,169,2329,132,2349,129
3,1,6001,202200,g22,06001202200,srprec,14,12,7,2,...,274,21,31,206,259,31,268,22,269,22
4,1,6001,202500,g22,06001202500,srprec,14,12,7,2,...,241,23,29,185,234,28,234,24,236,22


In [1495]:
## 2022 data engineering
cols = res_22.columns.values
pr_cols = cols[[x.startswith('PR_') for x in cols]]
pr_cols

array(['PR_1_N', 'PR_1_Y', 'PR_26_N', 'PR_26_Y', 'PR_27_N', 'PR_27_Y',
       'PR_28_N', 'PR_28_Y', 'PR_29_N', 'PR_29_Y', 'PR_30_N', 'PR_30_Y',
       'PR_31_N', 'PR_31_Y'], dtype=object)

In [1496]:
res_22 = res_22[np.append(pr_cols, 'SRPREC_KEY')]
res_22.head()

,PR_1_N,PR_1_Y,PR_26_N,PR_26_Y,PR_27_N,PR_27_Y,PR_28_N,PR_28_Y,PR_29_N,PR_29_Y,PR_30_N,PR_30_Y,PR_31_N,PR_31_Y,SRPREC_KEY
0,109,2372,1849,620,2260,264,391,2173,1604,859,1024,1516,284,2235,06001200100
1,85,1470,1151,416,1455,151,250,1356,1017,549,594,1000,205,1397,06001200200
2,104,2271,1686,672,2163,258,355,2100,1410,955,810,1643,298,2145,06001201400
3,24,252,173,103,230,50,46,237,150,123,73,215,58,224,06001202200
4,22,232,126,128,203,61,29,238,119,141,69,197,46,215,06001202500


In [1497]:
props = np.unique([re.sub("_Y", "", re.sub("_N", "", pr)) for pr in pr_cols])
props

array(['PR_1', 'PR_26', 'PR_27', 'PR_28', 'PR_29', 'PR_30', 'PR_31'],
      dtype='<U5')

In [1498]:
for pr in props:
    res_22[f'{pr}_TOT'] = res_22[f'{pr}_Y'] + res_22[f'{pr}_N']
res_22.head()

,PR_1_N,PR_1_Y,PR_26_N,PR_26_Y,PR_27_N,PR_27_Y,PR_28_N,PR_28_Y,PR_29_N,PR_29_Y,...,PR_31_N,PR_31_Y,SRPREC_KEY,PR_1_TOT,PR_26_TOT,PR_27_TOT,PR_28_TOT,PR_29_TOT,PR_30_TOT,PR_31_TOT
0,109,2372,1849,620,2260,264,391,2173,1604,859,...,284,2235,06001200100,2481,2469,2524,2564,2463,2540,2519
1,85,1470,1151,416,1455,151,250,1356,1017,549,...,205,1397,06001200200,1555,1567,1606,1606,1566,1594,1602
2,104,2271,1686,672,2163,258,355,2100,1410,955,...,298,2145,06001201400,2375,2358,2421,2455,2365,2453,2443
3,24,252,173,103,230,50,46,237,150,123,...,58,224,06001202200,276,276,280,283,273,288,282
4,22,232,126,128,203,61,29,238,119,141,...,46,215,06001202500,254,254,264,267,260,266,261


In [1499]:
## Crosswalk data
cw_22 = pd.read_csv('crosswalk/state_g22_sr_blk_map.csv')
cw_24 = pd.read_csv('crosswalk/state_g24_sr_blk_map.csv')
cw_22.head()

,COUNTY,BLOCK_KEY,FIPS,SRPREC,TRACT,SRPREC_KEY,ELECTION,GEO_TYPE,BLOCK,BLKREG,SRTOTREG,PCTSRPREC,BLKTOTREG,PCTBLK
0,1,60014211001000,6001,200100,421100,06001200100,g22,sr_blk,1000,56,3472.0,1.612903,56,100.0
1,1,60014211001001,6001,200100,421100,06001200100,g22,sr_blk,1001,42,3472.0,1.209677,42,100.0
2,1,60014211001002,6001,200100,421100,06001200100,g22,sr_blk,1002,86,3472.0,2.476959,86,100.0
3,1,60014211001003,6001,200100,421100,06001200100,g22,sr_blk,1003,110,3472.0,3.168203,110,100.0
4,1,60014211001004,6001,200100,421100,06001200100,g22,sr_blk,1004,92,3472.0,2.649770,92,100.0


In [1500]:
cw_24.head()

,COUNTY,BLOCK_KEY,FIPS,SRPREC,TRACT,SRPREC_KEY,ELECTION,GEO_TYPE,BLOCK,BLKREG,SRTOTREG,PCTSRPREC,BLKTOTREG,PCTBLK
0,1,60014211001000,6001,200100,421100,06001200100,g24,sr_blk,1000,68,3516.0,1.934016,68,100.0
1,1,60014211001001,6001,200100,421100,06001200100,g24,sr_blk,1001,54,3516.0,1.535836,54,100.0
2,1,60014211001002,6001,200100,421100,06001200100,g24,sr_blk,1002,96,3516.0,2.730375,96,100.0
3,1,60014211001003,6001,200100,421100,06001200100,g24,sr_blk,1003,122,3516.0,3.469852,122,100.0
4,1,60014211001004,6001,200100,421100,06001200100,g24,sr_blk,1004,89,3516.0,2.531286,89,100.0


In [1501]:
np.unique(cw_24['FIPS'])

array([6001, 6003, 6005, 6007, 6009, 6011, 6013, 6015, 6017, 6019, 6021,
       6023, 6025, 6027, 6029, 6031, 6033, 6035, 6037, 6039, 6041, 6043,
       6045, 6047, 6049, 6051, 6053, 6055, 6057, 6059, 6061, 6063, 6065,
       6067, 6069, 6071, 6073, 6075, 6077, 6079, 6081, 6083, 6085, 6087,
       6089, 6091, 6093, 6095, 6097, 6099, 6101, 6103, 6105, 6107, 6109,
       6111, 6113, 6115], dtype=int64)

In [1502]:
cw_24[cw_24['FIPS'] == 6039]

,COUNTY,BLOCK_KEY,FIPS,SRPREC,TRACT,SRPREC_KEY,ELECTION,GEO_TYPE,BLOCK,BLKREG,SRTOTREG,PCTSRPREC,BLKTOTREG,PCTBLK
159021,20,60390005094020,6039,1020,509,060391020,g24,sr_blk,4020,63,1664.0,3.786058,63,100.000000
159022,20,60390005132011,6039,2047,513,060392047,g24,sr_blk,2011,32,1897.0,1.686874,32,100.000000
159023,20,60390001032024,6039,5043,103,060395043,g24,sr_blk,2024,211,422.0,50.000000,211,100.000000
159024,20,60390007011025,6039,3024,701,060393024,g24,sr_blk,1025,48,1270.0,3.779528,48,100.000000
159025,20,60390008023010,6039,4039,802,060394039,g24,sr_blk,3010,28,1990.0,1.407035,28,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162031,20,60390005144003,6039,5181,514,060395181,g24,sr_blk,4003,1,1.0,100.000000,1,100.000000
162032,20,60390006032000,6039,4053,603,060394053,g24,sr_blk,2000,1,289.0,0.346021,214,0.467290
162033,20,60390001031083,6039,5042,103,060395042,g24,sr_blk,1083,1,6170.0,0.016207,72,1.388889
162034,20,60390001021153,6039,5042,102,060395042,g24,sr_blk,1153,1,6170.0,0.016207,1,100.000000


In [1503]:
cw_24['SRPREC_KEY'].value_counts().index.values.shape

(20084,)

In [1504]:
cons = pd.merge(left=cw_22, right=res_22, left_on='SRPREC_KEY', right_on='SRPREC_KEY', how='inner')
cons.head()

,COUNTY,BLOCK_KEY,FIPS,SRPREC,TRACT,SRPREC_KEY,ELECTION,GEO_TYPE,BLOCK,BLKREG,...,PR_30_Y,PR_31_N,PR_31_Y,PR_1_TOT,PR_26_TOT,PR_27_TOT,PR_28_TOT,PR_29_TOT,PR_30_TOT,PR_31_TOT
0,1,60014211001000,6001,200100,421100,06001200100,g22,sr_blk,1000,56,...,1516,284,2235,2481,2469,2524,2564,2463,2540,2519
1,1,60014211001001,6001,200100,421100,06001200100,g22,sr_blk,1001,42,...,1516,284,2235,2481,2469,2524,2564,2463,2540,2519
2,1,60014211001002,6001,200100,421100,06001200100,g22,sr_blk,1002,86,...,1516,284,2235,2481,2469,2524,2564,2463,2540,2519
3,1,60014211001003,6001,200100,421100,06001200100,g22,sr_blk,1003,110,...,1516,284,2235,2481,2469,2524,2564,2463,2540,2519
4,1,60014211001004,6001,200100,421100,06001200100,g22,sr_blk,1004,92,...,1516,284,2235,2481,2469,2524,2564,2463,2540,2519


In [1505]:
cons.columns.values

array(['COUNTY', 'BLOCK_KEY', 'FIPS', 'SRPREC', 'TRACT', 'SRPREC_KEY',
       'ELECTION', 'GEO_TYPE', 'BLOCK', 'BLKREG', 'SRTOTREG', 'PCTSRPREC',
       'BLKTOTREG', 'PCTBLK', 'PR_1_N', 'PR_1_Y', 'PR_26_N', 'PR_26_Y',
       'PR_27_N', 'PR_27_Y', 'PR_28_N', 'PR_28_Y', 'PR_29_N', 'PR_29_Y',
       'PR_30_N', 'PR_30_Y', 'PR_31_N', 'PR_31_Y', 'PR_1_TOT',
       'PR_26_TOT', 'PR_27_TOT', 'PR_28_TOT', 'PR_29_TOT', 'PR_30_TOT',
       'PR_31_TOT'], dtype=object)

In [1506]:
for pr in props:
    for suffix in ['_Y', '_N', '_TOT']:
        cons[f'{pr}{suffix}'] = cons[f'{pr}{suffix}'] * (cons['PCTSRPREC'] / 100)

In [1507]:
cons.head(3)

,COUNTY,BLOCK_KEY,FIPS,SRPREC,TRACT,SRPREC_KEY,ELECTION,GEO_TYPE,BLOCK,BLKREG,...,PR_30_Y,PR_31_N,PR_31_Y,PR_1_TOT,PR_26_TOT,PR_27_TOT,PR_28_TOT,PR_29_TOT,PR_30_TOT,PR_31_TOT
0,1,60014211001000,6001,200100,421100,06001200100,g22,sr_blk,1000,56,...,24.451613,4.580645,36.048387,40.016129,39.822581,40.709677,41.354839,39.725806,40.967742,40.629032
1,1,60014211001001,6001,200100,421100,06001200100,g22,sr_blk,1001,42,...,18.338710,3.435484,27.036290,30.012097,29.866935,30.532258,31.016129,29.794355,30.725806,30.471774
2,1,60014211001002,6001,200100,421100,06001200100,g22,sr_blk,1002,86,...,37.550691,7.034562,55.360023,61.453341,61.156106,62.518433,63.509217,61.007488,62.914747,62.394585


In [1508]:
cons = cons.groupby(['BLOCK_KEY'])[['PR_1_N', 'PR_1_Y', 'PR_26_N', 'PR_26_Y',
       'PR_27_N', 'PR_27_Y', 'PR_28_N', 'PR_28_Y', 'PR_29_N', 'PR_29_Y',
       'PR_30_N', 'PR_30_Y', 'PR_31_N', 'PR_31_Y', 'PR_1_TOT',
       'PR_26_TOT', 'PR_27_TOT', 'PR_28_TOT', 'PR_29_TOT', 'PR_30_TOT',
       'PR_31_TOT']].sum()
cons = cons.reset_index()
cons.head()

,BLOCK_KEY,PR_1_N,PR_1_Y,PR_26_N,PR_26_Y,PR_27_N,PR_27_Y,PR_28_N,PR_28_Y,PR_29_N,...,PR_30_Y,PR_31_N,PR_31_Y,PR_1_TOT,PR_26_TOT,PR_27_TOT,PR_28_TOT,PR_29_TOT,PR_30_TOT,PR_31_TOT
0,60014001001009,0.084806,1.295053,0.994700,0.355124,1.266784,0.113074,0.332155,1.060071,0.895760,...,0.839223,0.181979,1.201413,1.379859,1.349823,1.379859,1.392226,1.367491,1.401060,1.383392
1,60014001001010,4.452297,67.990283,52.221731,18.643993,66.506184,5.936396,17.438163,55.653710,47.027385,...,44.059187,9.553887,63.074205,72.442580,70.865724,72.442580,73.091873,71.793286,73.555654,72.628092
2,60014001001011,10.012689,103.011053,79.424369,32.726905,97.369654,16.647900,30.305938,84.841470,76.529099,...,63.484543,18.715876,95.159855,113.023742,112.151274,114.017554,115.147408,112.435967,114.792849,113.875730
3,60014001001013,0.508834,7.770318,5.968198,2.130742,7.600707,0.678445,1.992933,6.360424,5.374558,...,5.035336,1.091873,7.208481,8.279152,8.098940,8.279152,8.353357,8.204947,8.406360,8.300353
4,60014001001014,0.763251,11.655477,8.952297,3.196113,11.401060,1.017668,2.989399,9.540636,8.061837,...,7.553004,1.637809,10.812721,12.418728,12.148410,12.418728,12.530035,12.307420,12.609541,12.450530


In [1509]:
cons.shape

(376435, 22)

In [1510]:
np.unique(cons['BLOCK_KEY'].astype(str).map(lambda x: x[:4]))

array(['6001', '6003', '6005', '6007', '6009', '6011', '6013', '6015',
       '6017', '6019', '6021', '6023', '6025', '6027', '6029', '6031',
       '6033', '6035', '6037', '6039', '6041', '6043', '6045', '6047',
       '6049', '6051', '6053', '6055', '6057', '6059', '6061', '6063',
       '6065', '6067', '6069', '6071', '6073', '6075', '6077', '6079',
       '6081', '6083', '6085', '6087', '6089', '6091', '6093', '6095',
       '6097', '6099', '6101', '6103', '6105', '6107', '6109', '6111',
       '6113', '6115'], dtype=object)

In [1511]:
cons = pd.merge(left=cw_24, right=cons, on='BLOCK_KEY', how='right')
cons.head()

,COUNTY,BLOCK_KEY,FIPS,SRPREC,TRACT,SRPREC_KEY,ELECTION,GEO_TYPE,BLOCK,BLKREG,...,PR_30_Y,PR_31_N,PR_31_Y,PR_1_TOT,PR_26_TOT,PR_27_TOT,PR_28_TOT,PR_29_TOT,PR_30_TOT,PR_31_TOT
0,1.0,60014001001009,6001.0,213100,400100.0,06001213100,g24,sr_blk,1009.0,3.0,...,0.839223,0.181979,1.201413,1.379859,1.349823,1.379859,1.392226,1.367491,1.401060,1.383392
1,1.0,60014001001010,6001.0,213100,400100.0,06001213100,g24,sr_blk,1010.0,107.0,...,44.059187,9.553887,63.074205,72.442580,70.865724,72.442580,73.091873,71.793286,73.555654,72.628092
2,1.0,60014001001011,6001.0,213100,400100.0,06001213100,g24,sr_blk,1011.0,11.0,...,63.484543,18.715876,95.159855,113.023742,112.151274,114.017554,115.147408,112.435967,114.792849,113.875730
3,1.0,60014001001011,6001.0,213500,400100.0,06001213500,g24,sr_blk,1011.0,156.0,...,63.484543,18.715876,95.159855,113.023742,112.151274,114.017554,115.147408,112.435967,114.792849,113.875730
4,1.0,60014001001013,6001.0,213100,400100.0,06001213100,g24,sr_blk,1013.0,18.0,...,5.035336,1.091873,7.208481,8.279152,8.098940,8.279152,8.353357,8.204947,8.406360,8.300353


In [1512]:
# # Handle NaN
# cons_notna = cons[~cons['FIPS'].isna()]
# cons_na = cons[cons['FIPS'].isna()]
# cons_na['FIPS'] = cons_na['BLOCK_KEY'].map(lambda x: str(x)[:4])
# cons_notna['FIPS'] = cons_notna['BLOCK_KEY'].map(lambda x: str(x)[:4])
# cons_na.head()

In [1513]:
# cons = pd.concat([cons_na, cons_notna], axis=0)
# cons.shape

In [1514]:
# cons['SRPREC_KEY'] = '0' + cons['FIPS'].astype(int).astype(str) + cons['SRPREC'].astype(str)
# cons.head()

In [1515]:
for pr in props:
    for suffix in ['_Y', '_N', '_TOT']:
        cons[f'{pr}{suffix}'] = cons[f'{pr}{suffix}'] * (cons['PCTBLK'] / 100)

In [1516]:
cons.head(3)

,COUNTY,BLOCK_KEY,FIPS,SRPREC,TRACT,SRPREC_KEY,ELECTION,GEO_TYPE,BLOCK,BLKREG,...,PR_30_Y,PR_31_N,PR_31_Y,PR_1_TOT,PR_26_TOT,PR_27_TOT,PR_28_TOT,PR_29_TOT,PR_30_TOT,PR_31_TOT
0,1.0,60014001001009,6001.0,213100,400100.0,06001213100,g24,sr_blk,1009.0,3.0,...,0.839223,0.181979,1.201413,1.379859,1.349823,1.379859,1.392226,1.367491,1.401060,1.383392
1,1.0,60014001001010,6001.0,213100,400100.0,06001213100,g24,sr_blk,1010.0,107.0,...,44.059187,9.553887,63.074205,72.442580,70.865724,72.442580,73.091873,71.793286,73.555654,72.628092
2,1.0,60014001001011,6001.0,213100,400100.0,06001213100,g24,sr_blk,1011.0,11.0,...,4.181617,1.232782,6.268014,7.444678,7.387210,7.510138,7.584560,7.405962,7.561206,7.500797


In [1517]:
cw_24_eng = cons.copy()

In [1518]:
cons = cons.groupby(['SRPREC_KEY'])[['PR_1_N', 'PR_1_Y', 'PR_26_N', 'PR_26_Y',
       'PR_27_N', 'PR_27_Y', 'PR_28_N', 'PR_28_Y', 'PR_29_N', 'PR_29_Y',
       'PR_30_N', 'PR_30_Y', 'PR_31_N', 'PR_31_Y', 'PR_1_TOT',
       'PR_26_TOT', 'PR_27_TOT', 'PR_28_TOT', 'PR_29_TOT', 'PR_30_TOT',
       'PR_31_TOT']].sum()
cons = cons.reset_index()
cons.head()

,SRPREC_KEY,PR_1_N,PR_1_Y,PR_26_N,PR_26_Y,PR_27_N,PR_27_Y,PR_28_N,PR_28_Y,PR_29_N,...,PR_30_Y,PR_31_N,PR_31_Y,PR_1_TOT,PR_26_TOT,PR_27_TOT,PR_28_TOT,PR_29_TOT,PR_30_TOT,PR_31_TOT
0,610501.0,592.248503,905.071493,1042.697588,434.036958,1267.744413,218.980005,550.285113,944.551068,1182.280904,...,642.511213,634.922405,862.760932,1497.319996,1476.734546,1486.724418,1494.836181,1478.981405,1505.264558,1497.683337
1,610502.0,48.738579,72.774042,89.799162,29.042852,103.819849,16.023642,49.406231,70.771087,87.796207,...,53.078315,51.743012,69.101958,121.512621,118.842014,119.843492,120.177318,117.840536,121.512621,120.844969
2,610503.0,3.259398,4.971130,5.484182,2.401907,7.541002,0.366002,3.624750,4.637149,6.538411,...,3.446328,2.946331,4.929301,8.230528,7.886090,7.907004,8.261899,7.896547,8.261899,7.875633
3,610504.0,29.296059,29.377392,45.591035,13.088567,52.043074,7.038934,29.717630,29.562594,46.038279,...,19.936148,29.984933,28.502961,58.673452,58.679603,59.082008,59.280225,58.864983,59.273895,58.487893
4,610506.0,199.884049,240.457097,335.380504,101.656681,382.807493,57.578586,201.637985,239.260773,332.673043,...,169.605828,210.131437,229.241314,440.341146,437.037185,440.386078,440.898757,435.658106,443.120358,439.372750


In [1519]:
cons[cons['SRPREC_KEY'].map(lambda x: isinstance(x, float))]

,SRPREC_KEY,PR_1_N,PR_1_Y,PR_26_N,PR_26_Y,PR_27_N,PR_27_Y,PR_28_N,PR_28_Y,PR_29_N,...,PR_30_Y,PR_31_N,PR_31_Y,PR_1_TOT,PR_26_TOT,PR_27_TOT,PR_28_TOT,PR_29_TOT,PR_30_TOT,PR_31_TOT
0,610501.0,592.248503,905.071493,1042.697588,434.036958,1267.744413,218.980005,550.285113,944.551068,1182.280904,...,642.511213,634.922405,862.760932,1497.319996,1476.734546,1486.724418,1494.836181,1478.981405,1505.264558,1497.683337
1,610502.0,48.738579,72.774042,89.799162,29.042852,103.819849,16.023642,49.406231,70.771087,87.796207,...,53.078315,51.743012,69.101958,121.512621,118.842014,119.843492,120.177318,117.840536,121.512621,120.844969
2,610503.0,3.259398,4.971130,5.484182,2.401907,7.541002,0.366002,3.624750,4.637149,6.538411,...,3.446328,2.946331,4.929301,8.230528,7.886090,7.907004,8.261899,7.896547,8.261899,7.875633
3,610504.0,29.296059,29.377392,45.591035,13.088567,52.043074,7.038934,29.717630,29.562594,46.038279,...,19.936148,29.984933,28.502961,58.673452,58.679603,59.082008,59.280225,58.864983,59.273895,58.487893
4,610506.0,199.884049,240.457097,335.380504,101.656681,382.807493,57.578586,201.637985,239.260773,332.673043,...,169.605828,210.131437,229.241314,440.341146,437.037185,440.386078,440.898757,435.658106,443.120358,439.372750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643,61110097111.0,138.615854,313.336302,314.625752,133.780417,378.775879,69.952652,167.306112,283.356594,316.237564,...,186.647859,159.891775,291.738018,451.952155,448.406168,448.728531,450.662706,444.860181,451.307431,451.629793
1644,61110097112.0,149.923461,308.003379,301.788935,144.874225,390.733164,64.474856,154.584294,301.788935,315.383031,...,188.375333,150.311863,304.896157,457.926840,446.663160,455.208020,456.373229,449.381979,455.208020,455.208020
1645,61110097113.0,144.520138,232.264507,250.784945,120.231039,314.543830,62.544429,143.002069,231.050052,261.411426,...,144.520138,142.091228,234.389804,376.784645,371.015984,377.088259,374.052122,370.408757,377.391873,376.481032
1646,61110097114.0,109.962568,229.041203,219.355381,111.671830,284.877118,53.556898,124.776178,213.088084,225.337800,...,131.328351,124.776178,211.663699,339.003770,331.027211,338.434016,337.864262,333.875982,334.730614,336.439876


In [1520]:
cons.shape

(19971, 22)

In [1521]:
# def conv_srprec_str(srprec):
#     if isinstance(srprec, float):
#         prec = str(int(srprec))
#     else:
#         prec = srprec
#     return prec

In [1522]:
# cons['SRPREC_KEY'] = cons['SRPREC_KEY'].map(conv_srprec_str)
# cons.head()

In [1523]:
np.unique(cons['SRPREC_KEY'].astype(str).map(lambda x: x[:4]))

array(['0600', '0601', '0602', '0603', '0604', '0605', '0606', '0607',
       '0608', '0609', '0610', '6101', '6103', '6105', '6107', '6109',
       '6111', '6113', '6115'], dtype=object)

In [1524]:
np.unique(cons['SRPREC_KEY'].astype(str).map(lambda x: x[:5]))

array(['06001', '06003', '06005', '06007', '06009', '06011', '06013',
       '06015', '06017', '06019', '06021', '06023', '06025', '06027',
       '06029', '06031', '06033', '06035', '06037', '06039', '06041',
       '06043', '06045', '06047', '06049', '06051', '06053', '06055',
       '06057', '06059', '06061', '06063', '06065', '06067', '06069',
       '06071', '06073', '06075', '06077', '06079', '06081', '06083',
       '06085', '06087', '06089', '06091', '06093', '06095', '06097',
       '06099', '06101', '61010', '61030', '61050', '61051', '61071',
       '61072', '61073', '61074', '61075', '61091', '61092', '61093',
       '61094', '61095', '61110', '61131', '61158', '61159'], dtype=object)

In [1525]:
cw_24_eng = pd.DataFrame(index=cw_24_eng[['SRPREC_KEY', 'SRPREC', 'FIPS']].value_counts().index).reset_index()
cw_24_eng.head()

,SRPREC_KEY,SRPREC,FIPS
0,06071UNI0160,UNI0160,6071.0
1,06025483021,483021,6025.0
2,060395042,5042,6039.0
3,060311210,1210,6031.0
4,06071UNI0764,UNI0764,6071.0


In [1526]:
cons = pd.merge(left=cons, right=cw_24_eng, on='SRPREC_KEY', how='left')
cons['NEW_KEY'] = cons['FIPS'].astype(str) + cons['SRPREC'].astype(str)
cons.head()

,SRPREC_KEY,PR_1_N,PR_1_Y,PR_26_N,PR_26_Y,PR_27_N,PR_27_Y,PR_28_N,PR_28_Y,PR_29_N,...,PR_1_TOT,PR_26_TOT,PR_27_TOT,PR_28_TOT,PR_29_TOT,PR_30_TOT,PR_31_TOT,SRPREC,FIPS,NEW_KEY
0,610501.0,592.248503,905.071493,1042.697588,434.036958,1267.744413,218.980005,550.285113,944.551068,1182.280904,...,1497.319996,1476.734546,1486.724418,1494.836181,1478.981405,1505.264558,1497.683337,1.0,6105.0,6105.01.0
1,610502.0,48.738579,72.774042,89.799162,29.042852,103.819849,16.023642,49.406231,70.771087,87.796207,...,121.512621,118.842014,119.843492,120.177318,117.840536,121.512621,120.844969,2.0,6105.0,6105.02.0
2,610503.0,3.259398,4.971130,5.484182,2.401907,7.541002,0.366002,3.624750,4.637149,6.538411,...,8.230528,7.886090,7.907004,8.261899,7.896547,8.261899,7.875633,3.0,6105.0,6105.03.0
3,610504.0,29.296059,29.377392,45.591035,13.088567,52.043074,7.038934,29.717630,29.562594,46.038279,...,58.673452,58.679603,59.082008,59.280225,58.864983,59.273895,58.487893,4.0,6105.0,6105.04.0
4,610506.0,199.884049,240.457097,335.380504,101.656681,382.807493,57.578586,201.637985,239.260773,332.673043,...,440.341146,437.037185,440.386078,440.898757,435.658106,443.120358,439.372750,6.0,6105.0,6105.06.0


In [1527]:
#res_24 = pd.read_csv('data/state_g24_sov_data_by_g24_srprec.csv')
dbf = Dbf5("shapes/ca_24_allraces.dbf")
res_24 = dbf.to_dataframe()
res_24.head()

,SRPREC,COUNTY,FIPS_CODE,SRPREC_KEY,G24PDem,G24PRep,G24POth,G24PTot,G24SDem,G24SRep,...,G24PR3_T,G24PR4_N,G24PR4_Y,G24PR4_Y_2,G24PR5_N,G24PR5_Y,G24PR5_T,G24PR6_N,G24PR6_Y,G24PR6_T
0,491650,001,06001,06001491650,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,205200,001,06001,06001205200,3432,114,112,3658,3317,190,...,3561,478,3049,3527,1139,2319,3458,721,2700,3421
2,547720,001,06001,06001547720,1980,974,191,3145,1910,1079,...,3064,1349,1699,3048,2085,935,3020,1783,1157,2940
3,833300,001,06001,06001833300,1483,847,99,2429,1442,883,...,2323,846,1476,2322,1258,1041,2299,1172,1069,2241
4,333410,001,06001,06001333410,1792,317,108,2217,1800,309,...,2161,444,1704,2148,704,1427,2131,688,1407,2095


In [1528]:
# res_24[~res_24['SRPREC_KEY'].isin(cons['SRPREC_KEY'].values)]

In [1529]:
# cons[~cons['SRPREC_KEY'].isin(res_24['SRPREC_KEY'].values)]

In [1530]:
# cons['SRPREC_KEY'] = cons['SRPREC_KEY'].astype(str).str.rstrip('.0')
# cons[~cons['SRPREC_KEY'].isin(res_24['SRPREC_KEY'].values)]

In [1531]:
res_24.columns.values

array(['SRPREC', 'COUNTY', 'FIPS_CODE', 'SRPREC_KEY', 'G24PDem',
       'G24PRep', 'G24POth', 'G24PTot', 'G24SDem', 'G24SRep', 'G24STot',
       'G24SSDem', 'G24SSRep', 'G24SSTot', 'G24HDem', 'G24HRep',
       'G24HTot', 'G24SLDUDem', 'G24SLDURep', 'G24SLDUTot', 'G24SLDLDem',
       'G24SLDLRep', 'G24SLDLTot', 'G24PR2_N', 'G24PR2_Y', 'G24PR2_T',
       'G24PR32_N', 'G24PR32_Y', 'G24PR32_T', 'G24PR33_N', 'G24PR33_Y',
       'G24PR33_T', 'G24PR34_N', 'G24PR34_Y', 'G24PR34_T', 'G24PR35_N',
       'G24PR35_Y', 'G24PR35_T', 'G24PR36_N', 'G24PR36_Y', 'G24PR36_T',
       'G24PR3_N', 'G24PR3_Y', 'G24PR3_T', 'G24PR4_N', 'G24PR4_Y',
       'G24PR4_Y_2', 'G24PR5_N', 'G24PR5_Y', 'G24PR5_T', 'G24PR6_N',
       'G24PR6_Y', 'G24PR6_T'], dtype=object)

In [1539]:
# res_24['NEW_KEY'] = res_24['FIPS_CODE'].astype(str).map(lambda x: x[1:]) + res_24['SRPREC'].astype(str)
# res_24.head()

In [1549]:
res = pd.merge(left=cons, right=res_24, on='SRPREC_KEY', how='inner')
res.head()

,SRPREC_KEY,PR_1_N,PR_1_Y,PR_26_N,PR_26_Y,PR_27_N,PR_27_Y,PR_28_N,PR_28_Y,PR_29_N,...,G24PR3_T,G24PR4_N,G24PR4_Y,G24PR4_Y_2,G24PR5_N,G24PR5_Y,G24PR5_T,G24PR6_N,G24PR6_Y,G24PR6_T
0,06001200100,110.394301,2391.379349,1861.799846,628.266515,2280.397404,265.628899,393.789849,2191.582654,1613.914273,...,2920,433,2475,2908,1055,1792,2847,682,2101,2783
1,06001200200,84.131590,1476.736304,1153.935857,417.466221,1457.753361,152.717510,250.142427,1362.003012,1017.075146,...,2007,308,1681,1989,671,1264,1935,466,1454,1920
2,06001201400,103.578871,2247.796796,1672.623802,662.821849,2142.791281,254.711844,352.495938,2077.982694,1402.019259,...,2933,367,2547,2914,805,2059,2864,589,2225,2814
3,06001202200,23.911917,251.075125,172.365066,102.621975,229.155868,49.816493,45.831174,236.130177,149.449479,...,405,55,347,402,95,301,396,70,319,389
4,06001202500,21.736580,229.559095,124.916878,126.381705,200.960005,60.204696,28.722699,235.426705,117.895858,...,648,64,572,636,134,494,628,143,466,609


In [1552]:
cons[~cons['SRPREC'].isin([res_24['SRPREC'].values])]

,SRPREC_KEY,PR_1_N,PR_1_Y,PR_26_N,PR_26_Y,PR_27_N,PR_27_Y,PR_28_N,PR_28_Y,PR_29_N,...,PR_1_TOT,PR_26_TOT,PR_27_TOT,PR_28_TOT,PR_29_TOT,PR_30_TOT,PR_31_TOT,SRPREC,FIPS,NEW_KEY
0,610501.0,592.248503,905.071493,1042.697588,434.036958,1267.744413,218.980005,550.285113,944.551068,1182.280904,...,1497.319996,1476.734546,1486.724418,1494.836181,1478.981405,1505.264558,1497.683337,1.0,6105.0,6105.01.0
1,610502.0,48.738579,72.774042,89.799162,29.042852,103.819849,16.023642,49.406231,70.771087,87.796207,...,121.512621,118.842014,119.843492,120.177318,117.840536,121.512621,120.844969,2.0,6105.0,6105.02.0
2,610503.0,3.259398,4.971130,5.484182,2.401907,7.541002,0.366002,3.624750,4.637149,6.538411,...,8.230528,7.886090,7.907004,8.261899,7.896547,8.261899,7.875633,3.0,6105.0,6105.03.0
3,610504.0,29.296059,29.377392,45.591035,13.088567,52.043074,7.038934,29.717630,29.562594,46.038279,...,58.673452,58.679603,59.082008,59.280225,58.864983,59.273895,58.487893,4.0,6105.0,6105.04.0
4,610506.0,199.884049,240.457097,335.380504,101.656681,382.807493,57.578586,201.637985,239.260773,332.673043,...,440.341146,437.037185,440.386078,440.898757,435.658106,443.120358,439.372750,6.0,6105.0,6105.06.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19966,061010000020,149.600120,168.723909,213.723242,103.403496,245.362762,73.211175,133.712582,181.446540,238.271868,...,318.324029,317.126738,318.573938,315.159122,316.377314,317.605655,316.574970,0000020,6101.0,6101.00000020
19967,061010000021,61.262743,43.733550,83.263670,21.553753,89.971270,15.472196,57.059314,47.221501,81.206673,...,104.996293,104.817424,105.443466,104.280816,104.727989,105.085727,105.801205,0000021,6101.0,6101.00000021
19968,061010000022,49.603185,46.679618,72.504459,22.316561,81.664968,13.838217,47.751592,48.336306,77.279618,...,96.282803,94.821019,95.503185,96.087898,95.892994,95.795541,95.113376,0000022,6101.0,6101.00000022
19969,061010000023,15.939850,9.495166,20.193340,5.413534,21.783029,3.909774,16.197637,9.581096,21.740064,...,25.435016,25.606874,25.692803,25.778733,25.520945,25.692803,25.649839,0000023,6101.0,6101.00000023


In [1560]:
res.shape

(18250, 77)

In [1580]:
res.to_csv('transformed_data/all_results.csv')